In [31]:
import os.path
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import requests
import json




In [7]:
# Bittrex REST API
# GET = https://api.bittrex.com/v3
bittrex_rest = 'https://api.bittrex.com/v3'

# Binance REST API
# https://api.binance.com/api/v3 
binance_rest = 'https://api.binance.com/api/v3'
'https://api.binance.com/api/v3/exchangeInfo'



In [8]:
# GET /markets => https://api.bittrex.com/v3/markets
def bittrex_markets():
    t = eval(requests.get(bittrex_rest + '/markets').text)
    df = pd.DataFrame(t)
    df = df[df['status'] == 'ONLINE']
    df = df[df.astype(str)['prohibitedIn'] != '[\'US\']']
    
    return df

def bittrex_tickers():
    t = eval(requests.get(bittrex_rest + '/markets/tickers').text)
    df = pd.DataFrame(t)
    df_markets = bittrex_markets()
    df = df[df['symbol'].isin(df_markets['symbol'])]
    
    return df



def clean_hist_data(df):
    df.columns = ['datetime','open','high','low','close','volume','quoteVolume'] 
    df = df.astype({'open':'float64',
                    'high': 'float64', 
                    'low': 'float64', 
                    'close': 'float64', 
                    'volume': 'float64', 
                    'quoteVolume': 'float64' })
    df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True)
    return df

def bittrex_recent_candles(marketSymbol="ADA-USD",candleInterval="DAY_1"):
    # /markets/{marketSymbol}/candles/{candleType}/{candleInterval}/recent
    # marketSymbol arg: string ADA-USD
    # candleType may be omitted if trae based candles are desired
    # candleType args: string [TRADE, MIDPOINT]
    # candleInterval args: string [MINUTE_1, MINUTE_5, HOUR_1, DAY_1]
    
    t = requests.get(bittrex_rest + f'/markets/{marketSymbol}/candles/{candleInterval}/recent').json()
    
    df = pd.DataFrame(t)
    df = clean_hist_data(df)
    
    return df




#markets = bittrex_markets()

bittrex_recent_candles('ADA-USD','HOUR_1')





,datetime,open,high,low,close,volume,quoteVolume
0,2021-10-24 01:00:00,2.15648,2.16211,2.15331,2.15585,4133.375971,8917.454645
1,2021-10-24 02:00:00,2.15865,2.16000,2.15132,2.15701,43582.388858,94028.396505
2,2021-10-24 03:00:00,2.15540,2.16000,2.15540,2.15752,1360.342270,2935.207627
3,2021-10-24 04:00:00,2.15248,2.15512,2.14850,2.14930,11304.684196,24340.773033
4,2021-10-24 05:00:00,2.15060,2.15257,2.14667,2.14881,3685.881024,7920.752619
...,...,...,...,...,...,...,...
739,2021-11-23 20:00:00,1.78354,1.78354,1.76842,1.77175,129343.833466,229189.475638
740,2021-11-23 21:00:00,1.77151,1.77152,1.76237,1.76320,22310.656187,39373.911231
741,2021-11-23 22:00:00,1.76320,1.76647,1.75221,1.75739,191980.971543,337336.718310
742,2021-11-23 23:00:00,1.75802,1.76133,1.74144,1.75107,91929.831536,161097.040629


In [37]:
# GET test connectivity
def binance_ping():
    t = requests.get(binance_rest + r'/ping').json()
    return t

# GET /exchangeInfo => Current exchange trading rules and symbol information
# 'exchangeInfo'
# 'https://api.binance.com/api/v3/exchangeInfo'
# params = (('symbols', '["BTCUSDT","BNBBTC"]'),)
# response = requests.get('https://api.binance.com/api/v3/exchangeInfo', params=params)
def binance_markets():
    t = requests.get(binance_rest + r'/exchangeInfo').json()
    symbols = t['symbols']
    df = pd.DataFrame(symbols)
    
    return df

# GET /api/v3/klines
# params = (('symbols', '["BTCUSDT","BNBBTC"]'),)
# response = requests.get('https://api.binance.com/api/v3/exchangeInfo', params=params)
#[
#  [
#    1499040000000,      // Open time
#    "0.01634790",       // Open
#    "0.80000000",       // High
#    "0.01575800",       // Low
#    "0.01577100",       // Close
#    "148976.11427815",  // Volume
#    1499644799999,      // Close time
#    "2434.19055334",    // Quote asset volume
#    308,                // Number of trades
#    "1756.87402397",    // Taker buy base asset volume
#    "28.46694368",      // Taker buy quote asset volume
#    "17928899.62484339" // Ignore.
#  ]
#]
def binance_candles(symbol = 'BTCUSDT', interval = '1d'):
    params = (
        ('symbol', symbol),
        ('interval', interval),
             )
    t = requests.get(binance_rest + '/klines', params = params).json()
    
    
    return t



b_candles = binance_candles()
b_candles

[[1594771200000,
  '9255.85000000',
  '9276.49000000',
  '9160.57000000',
  '9197.60000000',
  '39053.57966500',
  1594857599999,
  '360201683.49480513',
  554780,
  '17977.67974700',
  '165822998.03166477',
  '0'],
 [1594857600000,
  '9197.60000000',
  '9226.15000000',
  '9047.25000000',
  '9133.72000000',
  '43375.57119100',
  1594943999999,
  '395883667.02480033',
  681807,
  '20410.13000900',
  '186271268.49276469',
  '0'],
 [1594944000000,
  '9133.72000000',
  '9186.83000000',
  '9089.81000000',
  '9154.32000000',
  '28054.35874100',
  1595030399999,
  '256360345.57676793',
  470991,
  '13397.53780500',
  '122434361.70789555',
  '0'],
 [1595030400000,
  '9154.31000000',
  '9219.30000000',
  '9121.10000000',
  '9170.28000000',
  '22554.54145700',
  1595116799999,
  '206628105.36278893',
  430377,
  '11502.70482900',
  '105387979.46391492',
  '0'],
 [1595116800000,
  '9170.30000000',
  '9232.27000000',
  '9101.35000000',
  '9208.99000000',
  '26052.01941700',
  1595203199999,
  '238

In [46]:

df_candles = pd.DataFrame(b_candles)

df_candles

,0,1,2,3,4,5,6,7,8,9,10,11
0,1594771200000,9255.85000000,9276.49000000,9160.57000000,9197.60000000,39053.57966500,1594857599999,360201683.49480513,554780,17977.67974700,165822998.03166477,0
1,1594857600000,9197.60000000,9226.15000000,9047.25000000,9133.72000000,43375.57119100,1594943999999,395883667.02480033,681807,20410.13000900,186271268.49276469,0
2,1594944000000,9133.72000000,9186.83000000,9089.81000000,9154.32000000,28054.35874100,1595030399999,256360345.57676793,470991,13397.53780500,122434361.70789555,0
3,1595030400000,9154.31000000,9219.30000000,9121.10000000,9170.28000000,22554.54145700,1595116799999,206628105.36278893,430377,11502.70482900,105387979.46391492,0
4,1595116800000,9170.30000000,9232.27000000,9101.35000000,9208.99000000,26052.01941700,1595203199999,238521988.03132549,440449,12873.37349300,117870221.42492930,0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,1637539200000,58617.70000000,59444.00000000,55610.00000000,56247.18000000,51724.32047000,1637625599999,2968622334.13765660,1624768,24834.98160000,1425668562.21166460,0
496,1637625600000,56243.83000000,58009.99000000,55317.00000000,57541.27000000,49917.85017000,1637711999999,2837210673.48851890,1570845,25217.65003000,1433171468.66686080,0
497,1637712000000,57541.26000000,57735.00000000,55837.00000000,57138.29000000,39612.04964000,1637798399999,2246374244.33056800,1432991,19625.09659000,1113184375.67946880,0
498,1637798400000,57138.29000000,59398.90000000,57000.00000000,58960.36000000,42153.51522000,1637884799999,2455658857.70059310,1604266,21112.80397000,1230022891.73009470,0
